In [15]:
import pandas as pd
pubmed = pd.read_csv('data/pubmed.csv')
pubmed.head()

,pubmed_id,title,keywords,journal,abstract,methods,results,conclusions,copyrights,doi,publication_date,authors
0,40050950,Efficacy of two doses of intra-articular ozone...,"Double-blind randomized controlled trial, Func...","Advances in rheumatology (London, England)","This double-blind, trial sought to assess the ...","This parallel, three-arm randomized controlled...",The groups receiving intra-articular ozone the...,Both 20 µg/mL and 40 µg/mL doses of intra-arti...,© 2025. The Author(s).,10.1186/s42358-025-00443-w\n10.1016/S0140-6736...,2025-03-07,"{'lastname': 'Arjmanddoust', 'firstname': 'Zah..."
1,40050948,Ovarian stimulation by promoting basal follicu...,"Advanced glycation end-products, Assisted repr...",Reproductive biology and endocrinology : RB&E,Most methods of ovarian stimulation rely on go...,NaN,While folliculogenesis following cyclic recrui...,Basal follicular growth modulation can increas...,© 2025. The Author(s).,10.1186/s12958-025-01356-5\n10.1210/er.2009-00...,2025-03-07,"{'lastname': 'Jinno', 'firstname': 'Masao', 'i..."
2,40050941,Physical activity and lung cancer screening (P...,"Exercise, Lung cancer, Prevention, Screening",Respiratory research,There is increasing evidence that screening pr...,NaN,75 participants were enrolled over a 3-month p...,NaN,© 2025. The Author(s).,10.1186/s12931-025-03158-0\n10.3322/caac.21660...,2025-03-07,"{'lastname': 'Bonney', 'firstname': 'Asha', 'i..."
3,40050934,GABA and GLP-1 receptor agonist combination th...,"Diabetes, GABA, GLP1-RA, Langerhans islet, Lir...",Diabetology & metabolic syndrome,Wolfram syndrome (WS) is a rare autosomal diso...,NaN,"Unlike in conventional diabetes models, GABA m...",GABA monotherapy had no significant effect on ...,© 2025. The Author(s).,10.1186/s13098-025-01651-6\n10.1136/jmg.34.10....,2025-03-07,"{'lastname': 'Jagomäe', 'firstname': 'Toomas',..."
4,40050933,Harnessing myeloid cells in cancer.,"Eosinophils, Immunosuppression, Macrophages, M...",Molecular cancer,Cancer-associated myeloid cells due to their p...,NaN,NaN,NaN,© 2025. The Author(s).,10.1186/s12943-025-02249-2\n10.1007/s00106-024...,2025-03-07,"{'lastname': 'Park', 'firstname': 'Su-Yeon', '..."


In [ ]:
import pandas as pd
import re
import json

# Function to extract NCT IDs safely
def extract_nct_ids_from_abstract(abstract):
    """
    Extract all NCT IDs from the abstract using regex.
    Ensure that the input is a string before applying regex.
    """
    if not isinstance(abstract, str):  # Ensure it's a valid string
        return []
    return re.findall(r'NCT\d+', abstract)

# Function to link PubMed articles to Clinical Trials
def link_pubmed_to_trials(pubmed_df):
    """
    Link PubMed articles to Clinical Trials using NCT IDs extracted from the abstracts.
    """
    data = []
    
    for index, row in pubmed_df.iterrows():
        pubmed_id = row['pubmed_id']
        abstract = row.get('abstract', '')  # Get abstract safely, default to empty string
        nct_ids = extract_nct_ids_from_abstract(abstract)
        
        if nct_ids:  # Only add if there are valid NCT IDs
            data.append({"PubMed_ID": pubmed_id, "NCT_IDs": nct_ids})
    
    return data

# Extracting and saving to JSON
linked_data = link_pubmed_to_trials(pubmed)

# Saving results to JSON
output_file = "linked_pubmed_nct_ids.json"
with open(output_file, 'w') as json_file:
    json.dump(linked_data, json_file, indent=4)

print(f"Data saved in {output_file}")


Data saved in linked_pubmed_nct_ids.json


In [26]:
import pandas as pd
import json

# pd.read_csv('data/all_diseas_processed.csv')
all_disease = pd.read_csv('data/all_disease_procesed.csv')
print(all_disease.head())

pubmed_links = json.load(open('linked_pubmed_nct_ids.json'))
print(pubmed_links[:5])

        NCT ID  Acronym           Overall Status  Start Date  \
0  NCT02695966     EVIS                COMPLETED     2015-05   
1  NCT03882866  Unknown                  UNKNOWN  2019-03-28   
2  NCT04572165  Unknown  ENROLLING_BY_INVITATION  2021-01-26   
3  NCT03190265  Unknown                COMPLETED  2017-12-14   
4  NCT02822066  Unknown                  UNKNOWN     2016-04   

                  Conditions  \
0          Pancreatic Cancer   
1          Pancreatic Cancer   
2  Diabetes Mellitus, Type 2   
3          Pancreatic Cancer   
4       Refractory Neoplasms   

                                       Interventions  \
0                                   Whipples Surgery   
1  3D-printed non-coplanar template, 3D-printed c...   
2             Ozempic®, Active Comparator, Rybelsus®   
3  Cyclophosphamide, Nivolumab, Ipilimumab, GVAX ...   
4                                    Electroporation   

                                           Locations Primary Completion Date  \
0    

In [27]:


# Create a mapping of NCT ID to PubMed ID
nct_to_pubmed = {}

for entry in pubmed_links:
    pubmed_id = str(entry["PubMed_ID"])  # Ensure string format
    for nct_id in entry["NCT_IDs"]:
        nct_to_pubmed[nct_id] = pubmed_id

# Sample all_disease DataFrame
# all_disease = pd.DataFrame({
#     "NCT ID": ["NCT06088706", "NCT02871856", "NCT12345678", "NCT05622630"]
# })

# Map NCT ID to PubMed ID
all_disease["Associated Article ID"] = all_disease["NCT ID"].map(nct_to_pubmed).fillna("")

# Create binary column indicating association
all_disease["Associated Article?"] = all_disease["Associated Article ID"].apply(lambda x: "YES" if x else "NO")

# Display result
print(all_disease)


           NCT ID  Acronym           Overall Status  Start Date  \
0     NCT02695966     EVIS                COMPLETED     2015-05   
1     NCT03882866  Unknown                  UNKNOWN  2019-03-28   
2     NCT04572165  Unknown  ENROLLING_BY_INVITATION  2021-01-26   
3     NCT03190265  Unknown                COMPLETED  2017-12-14   
4     NCT02822066  Unknown                  UNKNOWN     2016-04   
...           ...      ...                      ...         ...   
3631  NCT00592553  Unknown                COMPLETED  2008-02-29   
3632  NCT00102453  Unknown                COMPLETED     2002-03   
3633  NCT01350154  Unknown                COMPLETED     2011-11   
3634  NCT00016653  Unknown                COMPLETED     2000-06   
3635  NCT02653833  Unknown               TERMINATED  2017-11-01   

                                             Conditions  \
0                                     Pancreatic Cancer   
1                                     Pancreatic Cancer   
2                 

In [28]:
all_disease.head()

,NCT ID,Acronym,Overall Status,Start Date,Conditions,Interventions,Locations,Primary Completion Date,Study First Post Date,Last Update Post Date,Study Type,Phases,Sponsor,Sponsor Type,Disease,Associated Article ID,Associated Article?
0,NCT02695966,EVIS,COMPLETED,2015-05,Pancreatic Cancer,Whipples Surgery,Cambridge - United Kingdom,2018-07,2016-03-02,2019-07-16,OBSERVATIONAL,Not Available,Cambridge University Hospitals NHS Foundation ...,OTHER,pancreatic_cancer,,NO
1,NCT03882866,Unknown,UNKNOWN,2019-03-28,Pancreatic Cancer,"3D-printed non-coplanar template, 3D-printed c...","Hefei - China, Beijing - China, Guangzhou - Ch...",2021-06-30,2019-03-20,2020-02-06,INTERVENTIONAL,NaN,Ruijin Hospital,OTHER,pancreatic_cancer,,NO
2,NCT04572165,Unknown,ENROLLING_BY_INVITATION,2021-01-26,"Diabetes Mellitus, Type 2","Ozempic®, Active Comparator, Rybelsus®",Odense - Denmark,2024-12-31,2020-10-01,2024-01-02,OBSERVATIONAL,Not Available,Novo Nordisk A/S,INDUSTRY,pancreatic_cancer,,NO
3,NCT03190265,Unknown,COMPLETED,2017-12-14,Pancreatic Cancer,"Cyclophosphamide, Nivolumab, Ipilimumab, GVAX ...",Baltimore - United States,2022-08-03,2017-06-16,2024-08-07,INTERVENTIONAL,PHASE2,Sidney Kimmel Comprehensive Cancer Center at J...,OTHER,pancreatic_cancer,,NO
4,NCT02822066,Unknown,UNKNOWN,2016-04,Refractory Neoplasms,Electroporation,Hangzhou - China,2018-06,2016-07-04,2016-07-04,INTERVENTIONAL,PHASE2,First Affiliated Hospital of Zhejiang University,OTHER,pancreatic_cancer,,NO
